## Clasificación de documentos:

# Calculo del género de una película a raíz de su sinopsis

## Parte I: Introducción

Se requiere la implementación de un algoritmo que, previamente entrenado, calcule el __género principal__ de una película en base al conocimiento adquirido.

Los géneros que tendrá en cuenta el algoritmo son: ___acción___, ___comedia___, ___terror___, ___bélico___.

Los __archivos que contienen la sinopsis__ (o equivalente, pudiendo ser también un breve resumen de la primera parte de la película) de las películas estarán distribuídos de la siguiente forma: 1) Si forman parte del conjunto de pruebas estarán dentro de la carpeta del conjunto de prueba sin más, que será la carpeta en la que el algoritmo, una vez entrenado, buscará sinopsis para categorizarlas 2) Si forman parte del conjunto de entrenamiento estarán dentro de la carpeta del conjunto de entrenamiento y __a su vez__ dentro de una carpeta que indique su género.

El __formato__ de los archivos será _.txt_, por lo tanto estamos hablando de texto plano.

__Los conocimientos requeridos por parte del usuario__ que ejecutará el algoritmo __son mínimos__: tan sólo necesita __colocar__ los textos, en el formato adecuado, en la carpeta indicada y __ejecutar__ el algoritmo en sí. Únicamente se requiere mayor interacción por parte del usuario si desea cambiar el funcionamiento del algorimo en sí, o ajustar el mismo, como por ejemplo si desea cambiar las palabras clave para cada una de las categorías.

## Parte II: Escaneo de archivos

Empezamos símplemente indicando qué __carpeta__ contiene el __conjunto de entrenamiento__ y el de __prueba__, además de las __categorías__, que no serán útiles más adelante.

In [252]:
ruta_conjunto_entrenamiento = "conjunto_entrenamiento"
ruta_conjunto_prueba = "conjunto_prueba"

categorías = {"acción", "comedia", "terror", "bélico"}

A continuación, vamos a recorrer nuestra estructura de carpetas para detectar cada uno de los textos a analizar.

Primero __empezamos encontrando__ todos los __archivos ya clasificados__ (conjunto de entrenamiento), según su __categoría__.

In [253]:
import os # Nos ayudaremos de la librería "os" para leer ficheros y carpetas.

# NOTA: Vamos a almacenarlos en conjunto (set) ya que no nos interesa el orden y, además, no permite duplicados (no puede haber dos archivos con el mismo nombre).

total_documentos_entrenamiento = 0 # Util en la parte 4 ("Procesamiento")

archivos_entrenamiento_acción = set()
total_documentos_acción_entrenamiento = 0 # Util en la parte 4 ("Procesamiento")
for file in os.listdir(ruta_conjunto_entrenamiento + "/" + "acción"):
    if file.endswith(".txt"):
        aux = "acción" + "/" + file
        archivos_entrenamiento_acción.add(aux)
        total_documentos_entrenamiento +=1 # Util en la parte 4 ("Procesamiento")
        total_documentos_acción_entrenamiento += 1 # Util en la parte 4 ("Procesamiento")

archivos_entrenamiento_comedia = set()
total_documentos_comedia_entrenamiento = 0 # Util en la parte 4 ("Procesamiento")
for file in os.listdir(ruta_conjunto_entrenamiento + "/" + "comedia"):
    if file.endswith(".txt"):
        aux = "comedia" + "/" + file
        archivos_entrenamiento_comedia.add(aux)
        total_documentos_entrenamiento +=1 # Util en la parte 4 ("Procesamiento")
        total_documentos_comedia_entrenamiento += 1 # Util en la parte 4 ("Procesamiento")

archivos_entrenamiento_terror = set()
total_documentos_terror_entrenamiento = 0 # Util en la parte 4 ("Procesamiento")
for file in os.listdir(ruta_conjunto_entrenamiento + "/" + "terror"):
    if file.endswith(".txt"):
        aux = "terror" + "/" + file
        archivos_entrenamiento_terror.add(aux)
        total_documentos_entrenamiento +=1 # Util en la parte 4 ("Procesamiento")
        total_documentos_terror_entrenamiento += 1 # Util en la parte 4 ("Procesamiento")

archivos_entrenamiento_bélico = set()
total_documentos_bélico_entrenamiento = 0 # Util en la parte 4 ("Procesamiento")
for file in os.listdir(ruta_conjunto_entrenamiento + "/" + "bélico"):
    if file.endswith(".txt"):
        aux = "bélico" + "/" + file
        archivos_entrenamiento_bélico.add(aux)
        total_documentos_entrenamiento +=1 # Util en la parte 4 ("Procesamiento")
        total_documentos_bélico_entrenamiento += 1 # Util en la parte 4 ("Procesamiento")

Y, a continuación, procedemos a __encontrar__ los __archivos que querríamos clasificar__ (conjunto de prueba):

In [254]:
# Usamos conjunto por la misma razón de arriba.

archivos_prueba = set()
for file in os.listdir(ruta_conjunto_prueba):
    if file.endswith(".txt"):
        archivos_prueba.add(file)

print(archivos_prueba)

{'Teléfono rojo - Volamos hacia Moscú.txt', 'El padrino.txt', 'La caza del Octubre Rojo.txt', 'El tirador.txt'}


## Parte III: Elección del conjunto de palabras clave

Para ayudarnos en el estudio de las __palabras clave__ que debemos escoger para cada categoría vamos a realizar un pequeño estudio para determinar las palabras más frecuentes de cada categoría. La elección en sí debemos realizarla a mano puesto que no podemos escoger directamente las más frecuentes puesto que con total seguridad entre las más frecuentes se encontrarán verbos, conectores, preposiciones, pronombres, artículos, nombres propios, etc. que no nos serán de utilidad a la hora de determinar la categoría de una película.

Debemos definir __método que recibirá__ tanto __un conjunto de archivos__ como una __ruta__ donde se encuentran y __contará las palabras__ que aparecen en él __y el número de veces que dichas palabras aparecen__.

Pero antes, definimos un método que reciba un solo archivo y cuente sus palabras. Este metodo será usado por el método que estamos buscando y los dividimos de esta forma porque debemos buscar en un solo archivo cuando apliquemos los algoritmos de Naive Bayes y kNN.

In [255]:
def cuenta_palabras_desde_archivo(ruta, archivo):
    cuenta_palabras = {}
    
    file = open(ruta + "/" + archivo, "r", encoding="latin-1") # Elegimos latin-1 en vez de utf-8 por problemas con las tildes.

    for palabra in file.read().split(): # Recorremos el fichero, palabra a palabra.
        palabra = palabra.lower() # Pasamos la palabra a minúscula.
        # Para perder la menor información posible, reemplazamos ':', ',', ':' y ';', que son los carácteres más típicos que nos podemos encontrar adyacentes a una palabra y que la invalidarían en el siguiente if del algoritmo.
        palabra = palabra.replace('.', '')
        palabra = palabra.replace(',', '')
        palabra = palabra.replace(':', '')
        palabra = palabra.replace(';', '')
        if palabra.isalpha() is True: # Será true cuando todos los caracteres son alfabéticos y hay al menos uno.
            if palabra in cuenta_palabras:
                cuenta_palabras[palabra] += 1 # Si la palabra ya existe, entonces incrementa en 1 el número de veces que hace aparición.
            else:
                cuenta_palabras[palabra] = 1 # Si la palabra no existe, la añade (con valor 1 al número de veces que aparece).

    cuenta_palabras
    return cuenta_palabras

Ahora sí, definimos el método que estamos buscando y que se anunciaba antes:

In [256]:
from collections import Counter # Lo usaremos para añadir un diccionario a otro.

def cuenta_palabras_desde_archivos(ruta, archivos):
    cuenta_palabras = {}
    
    for archivo in archivos:
        new = cuenta_palabras_desde_archivo(ruta, archivo)
        cuenta_palabras = dict(Counter(cuenta_palabras)+Counter(new))
    
    return cuenta_palabras

Además, con propósito de limpiar el output que obtendremos vamos a establecer un __umbral__ para desechar todas las palabras que se repitan por debajo del mismo:

In [257]:
umbral_repetición = 4

Y, con el mismo propósito, otro __umbral__ para desechar todas las palabras con una longitud menor a él:

In [258]:
umbral_longitud = 4

Y, por último, un método que use al anterior que, además, nos __ordene las palabras__ (de mayor a menor uso) __y las muestre__:

In [259]:
def cuenta_palabras_desde_archivos_ordenadas(ruta, archivos):
    cuenta_palabras = cuenta_palabras_desde_archivos(ruta, archivos)
    
    lista = [] # Usamos una lista para poder ordenar las palabras (map).
    for palabra, contador in cuenta_palabras.items():
        lista.append((contador, palabra))

    lista = sorted(lista, reverse = True) # Lo ordenamos y lo invertimos para que las palabras más frecuentes estén arriba.

    for aux in lista:
        if(aux[0] >= umbral_repetición and len(aux[1]) >= umbral_longitud): # Si la palabra supera los umbrales indicados, se muestra.
            print("%s \t %d" % (aux[1], aux[0]))

Palabras más frecuentes en: __ACCIÓN__.

In [260]:
cuenta_palabras_desde_archivos_ordenadas(ruta_conjunto_entrenamiento, archivos_entrenamiento_acción)

para 	 75
brian 	 35
pero 	 30
después 	 30
mientras 	 28
policía 	 25
ethan 	 24
bryan 	 24
está 	 23
mcclane 	 21
donde 	 21
quien 	 19
como 	 19
tiene 	 18
equipo 	 17
coche 	 17
agente 	 17
letty 	 16
hunt 	 15
dice 	 15
auto 	 15
ellos 	 14
ella 	 14
cuando 	 14
entonces 	 13
encuentra 	 13
embargo 	 13
novia 	 12
martin 	 12
dominic 	 12
cual 	 12
toretto 	 11
sobre 	 11
luego 	 11
john 	 11
hombres 	 11
también 	 10
llega 	 10
hombre 	 10
escapar 	 10
carrera 	 10
bourne 	 10
ahora 	 10
tras 	 9
sean 	 9
nueva 	 9
logra 	 9
lleva 	 9
grupo 	 9
contra 	 9
antes 	 9
aeropuerto 	 9
vida 	 8
unos 	 8
todo 	 8
película 	 8
owen 	 8
momento 	 8
lugar 	 8
hija 	 8
hasta 	 8
había 	 8
están 	 8
descubre 	 8
virus 	 7
unidos 	 7
todos 	 7
tego 	 7
tarde 	 7
tanto 	 7
sigue 	 7
shaw 	 7
seguridad 	 7
reúne 	 7
plan 	 7
mills 	 7
llegan 	 7
llamada 	 7
lenore 	 7
información 	 7
informa 	 7
gasolina 	 7
gana 	 7
estados 	 7
esposa 	 7
durante 	 7
desde 	 7
comienza 	 7
clay 	 7
ciudad 	 7


Palabras más frecuentes en: __COMEDIA__.

In [261]:
cuenta_palabras_desde_archivos_ordenadas(ruta_conjunto_entrenamiento, archivos_entrenamiento_comedia)

tras 	 8
museo 	 8
para 	 6
kahmunrah 	 6
tabla 	 4
siendo 	 4
larry 	 4
jedediah 	 4
figuras 	 4
ellos 	 4
ello 	 4
como 	 4
ayuda 	 4


Palabras más frecuentes en: __TERROR__.

In [262]:
cuenta_palabras_desde_archivos_ordenadas(ruta_conjunto_entrenamiento, archivos_entrenamiento_terror)

lorraine 	 17
casa 	 17
carolyn 	 17
aunque 	 15
cuando 	 12
april 	 11
tras 	 10
ella 	 10
alguien 	 10
todo 	 8
mujer 	 8
donde 	 8
sobre 	 7
siendo 	 7
para 	 7
habitación 	 7
exorcismo 	 7
espíritu 	 7
warren 	 6
roger 	 6
muñeca 	 6
cómo 	 6
armario 	 6
viendo 	 5
ruidos 	 5
observando 	 5
noche 	 5
mientras 	 5
judy 	 5
hija 	 5
hacia 	 5
está 	 5
como 	 5
allí 	 5
todos 	 4
tiene 	 4
sótano 	 4
suelo 	 4
siguiente 	 4
rory 	 4
puerta 	 4
padre 	 4
lugar 	 4
hijo 	 4
hijas 	 4
familia 	 4
escucha 	 4
entretanto 	 4
encontrando 	 4
dice 	 4
descubre 	 4
debe 	 4
cual 	 4
contra 	 4
consigue 	 4
christine 	 4
annabelle 	 4


Palabras más frecuentes en: __BÉLICO__.

In [263]:
cuenta_palabras_desde_archivos_ordenadas(ruta_conjunto_entrenamiento, archivos_entrenamiento_bélico)

para 	 44
guerra 	 39
como 	 24
ejército 	 22
sargento 	 21
soldados 	 19
está 	 18
durante 	 18
general 	 16
cuando 	 16
grupo 	 14
sobre 	 12
hombres 	 12
hitler 	 12
combate 	 12
este 	 11
tres 	 10
tras 	 10
todo 	 10
patoso 	 10
misión 	 10
mientras 	 10
después 	 10
ante 	 10
unidos 	 9
quien 	 9
pero 	 9
matar 	 9
estados 	 9
contra 	 9
teniente 	 8
stauffenberg 	 8
pelotón 	 8
james 	 8
ellos 	 8
capitán 	 8
alemán 	 8
alemanes 	 8
vietnam 	 7
tiene 	 7
tiempo 	 7
solo 	 7
recibe 	 7
raine 	 7
policías 	 7
parte 	 7
japonés 	 7
isla 	 7
historia 	 7
estadounidenses 	 7
estadounidense 	 7
entre 	 7
edificio 	 7
donde 	 7
coronel 	 7
compañeros 	 7
comienza 	 7
batalla 	 7
albert 	 7
vietnamita 	 6
todos 	 6
también 	 6
soldado 	 6
sido 	 6
regimiento 	 6
primera 	 6
pesar 	 6
otros 	 6
ordena 	 6
operación 	 6
oficial 	 6
nuevo 	 6
norman 	 6
marina 	 6
mando 	 6
kuribayashi 	 6
infantería 	 6
hombre 	 6
están 	 6
equipo 	 6
desde 	 6
bajo 	 6
algren 	 6
zaitsev 	 5
vida 	 5
tro

Una vez realizado el estudio, inicializamos manualmente las __palabras clave__ de cada __categoría__, usaremos unas 20 palabras para cada una de ellas (podrán repetirse entre categorías).

In [264]:
# Las inicializamos como conjuntos ya que no nos interesa el orden y, además, no se permiten duplicados.
palabras_clave_acción = {"policía", "coche", "agente", "auto", "escapar", "carrera", "seguridad", "llamada", "gasolina", "problemas", "operación", "escapa", "venganza", "prisión", "muerte", "hospital", "carreras", "ataque", "arma", "asesino", "asesina", "asesinos", "terrorista", "terroristas", "persecución", "muerto", "investigación", "destrucción", "bomba"}
palabras_clave_comedia = {}
palabras_clave_terror = {}
palabras_clave_bélico = {"guerra", "ejército", "sargento", "soldados", "general", "grupo", "hombres", "combate", "misión", "unidos", "matar", "teniente", "capitán", "alemán", "alemanes", "vietnam", "estadounidense", "estadounidenses", "soldado", "regimiento", "operación", "oficial", "infantería", "cabo", "enemigo"}

Y, a continuación, añadimos todas las palabras clave de cada categoría a un nuevo conjunto que contenga __todas las palabras clave__.

In [265]:
palabras_clave = set()
# Update nos permite añadir el contenido de un set a otro set
palabras_clave.update(palabras_clave_acción)
palabras_clave.update(palabras_clave_comedia)
palabras_clave.update(palabras_clave_terror)
palabras_clave.update(palabras_clave_bélico)
print("Las palabras clave son: %s" % (palabras_clave))

Las palabras clave son: {'alemán', 'escapa', 'bomba', 'venganza', 'problemas', 'sargento', 'operación', 'oficial', 'enemigo', 'policía', 'cabo', 'carrera', 'asesino', 'estadounidenses', 'misión', 'investigación', 'vietnam', 'teniente', 'asesinos', 'combate', 'ejército', 'auto', 'guerra', 'escapar', 'seguridad', 'terrorista', 'carreras', 'soldado', 'grupo', 'persecución', 'hospital', 'muerto', 'soldados', 'alemanes', 'general', 'hombres', 'infantería', 'regimiento', 'unidos', 'asesina', 'destrucción', 'ataque', 'estadounidense', 'llamada', 'terroristas', 'prisión', 'agente', 'arma', 'coche', 'muerte', 'gasolina', 'capitán', 'matar'}


## Parte IV: Procesamiento

En esta parte se va a llevar a cabo el procesamiento de datos para posteriormente utilizarlos en los algoritmos de __Naive Bayes__ y __kNN__.

Puesto que se deben realizar cálculos distintos para cada algoritmo, dividiremos esta sección en dos subsecciones:

### Parte IV-A: Procesamiento de Naive Bayes

Para aplicar el algoritmo Naive Bayes primero debemos calcular todos los __P(c)__ ___(probabilidad de "c")___ y los __P(t|c)__ ___(probabilidad de "t" condicionada a "c")___.

En este caso __"c"__ sería nuestra categoría y __"t"__ cada palabra clave.

Primero, vamos a calcular los __P(c)__. Para ello, tenemos que contar el número de documentos de la categoría en cuestión existentes en nuestro conjunto de entrenamiento y dividirlo entre el número total de documentos de nuestro conjunto de entrenamiento. Así pues, por ejemplo, la probabilidad de acción (__P(acción)__) sería el número resultante de dividir el total de documentos catalogados como "acción" de nuestro conjunto de entrenamiento entre el número total de documentos del conjunto de entrenamiento.

Para ahorrarnos repetir los mismos bucles, los cálculos necesarios para elaborar ahora los __P(c)__ ya se han realizado en la parte I del documento, como se indicaba en el código de dicha parte: Allí se calculaban el total de documentos y el total de documentos de cada categoría.

In [266]:
probabilidad_acción = total_documentos_acción_entrenamiento / total_documentos_entrenamiento
print("P(acción) = \t %f" % (probabilidad_acción))

P(acción) = 	 0.530612


In [267]:
probabilidad_comedia = total_documentos_comedia_entrenamiento / total_documentos_entrenamiento
print("P(comedia) = \t %f" % (probabilidad_comedia))

P(comedia) = 	 0.020408


In [268]:
probabilidad_terror = total_documentos_terror_entrenamiento / total_documentos_entrenamiento
print("P(terror) = \t %f" % (probabilidad_terror))

P(terror) = 	 0.020408


In [269]:
probabilidad_bélico = total_documentos_bélico_entrenamiento / total_documentos_entrenamiento
print("P(bélico) = \t %f" % (probabilidad_bélico))

P(bélico) = 	 0.428571


Sólo para asegurarnos, todas las probabilidades deben sumar __~1__ en este apartado:

In [270]:
print("Suma de probabilidades (debe ser ~1) \t %f" % (probabilidad_acción + probabilidad_comedia + probabilidad_terror + probabilidad_bélico))

Suma de probabilidades (debe ser ~1) 	 1.000000


Ahora, para calcular los __P(t|c)__ será un poco más complejo. Para llevar a cabo esta tarea haremos uso de __un diccionario por cada categoría__ que __relacionará palabras clave con su probabilidad condicionada a la categoría del diccionario__. De nuevo, tomaremos la categoría "acción" como ejemplo: el diccionario "probabilidad_palabraclave_acción" recogerá, por cada palabra clave, su probabilidad condiccionada a la categoría acción, es decir, su __P(t|acción__), siendo "t" cada entrada del diccionario.

Pero antes de empezar, definiremos un método, que utilizaremos en los siguientes pasos, que genere cada diccionario deseado como salida y, además, le __aplique un suavizado de LaPlace__:

In [271]:
def crea_diccionario_probabilidades_condicionadas(archivo_entrenamiento_categoría, palabras_clave_categoría):
    probabilidades_condicionadas = {}
    cuenta_palabras_categoría = cuenta_palabras_desde_archivos(ruta_conjunto_entrenamiento, archivo_entrenamiento_categoría) # Almacena las palabras clave de la categoría y el número de veces que se repiten.
    
    número_palabras_clave_totales = len(palabras_clave) # Número de palabras clave que poseemos en total.
    número_palabras_clave_categoría = sum(cuenta_palabras_categoría.values())
    
    for palabra_clave in palabras_clave:
        if palabra_clave in palabras_clave_categoría:
            número_veces_aparece_palabra_en_categoría = cuenta_palabras_categoría[palabra_clave] # Número de veces que la palabra clave se repite en esta categoría.
        else:
            número_veces_aparece_palabra_en_categoría = 0
        result = ((número_veces_aparece_palabra_en_categoría + 1) / (número_palabras_clave_categoría + número_palabras_clave_totales)) # Añadimos 1 en el numerador y el número de palabras claves totales en el denominador para aplicar el suavizado.
        probabilidades_condicionadas[palabra_clave] = result
    
    return probabilidades_condicionadas

También vamos a crear un pequeño método que nos ayude a visualizar las probabilidades condicionadas:

In [272]:
def mostrar_diccionario_probabilidades_condicionadas(diccionario, categoría):
    for entrada in diccionario:
        print("P(%s|%s) = \t %f" % (entrada, categoría, diccionario[entrada]))

Ahora, empezamos con el cálculo en sí:

In [273]:
probabilidad_palabraclave_acción = crea_diccionario_probabilidades_condicionadas(archivos_entrenamiento_acción, palabras_clave_acción)

mostrar_diccionario_probabilidades_condicionadas(probabilidad_palabraclave_acción, "acción")

P(alemán|acción) = 	 0.000129
P(escapa|acción) = 	 0.000900
P(bomba|acción) = 	 0.000643
P(venganza|acción) = 	 0.000772
P(problemas|acción) = 	 0.000900
P(sargento|acción) = 	 0.000129
P(operación|acción) = 	 0.000900
P(oficial|acción) = 	 0.000129
P(enemigo|acción) = 	 0.000129
P(policía|acción) = 	 0.003344
P(cabo|acción) = 	 0.000129
P(carrera|acción) = 	 0.001415
P(asesino|acción) = 	 0.000514
P(estadounidenses|acción) = 	 0.000129
P(misión|acción) = 	 0.000129
P(investigación|acción) = 	 0.000643
P(vietnam|acción) = 	 0.000129
P(teniente|acción) = 	 0.000129
P(asesinos|acción) = 	 0.000772
P(combate|acción) = 	 0.000129
P(ejército|acción) = 	 0.000129
P(auto|acción) = 	 0.002058
P(guerra|acción) = 	 0.000129
P(escapar|acción) = 	 0.001415
P(seguridad|acción) = 	 0.001029
P(terrorista|acción) = 	 0.000643
P(carreras|acción) = 	 0.000772
P(soldado|acción) = 	 0.000129
P(grupo|acción) = 	 0.000129
P(persecución|acción) = 	 0.000643
P(hospital|acción) = 	 0.000772
P(muerto|acción) = 

In [274]:
probabilidad_palabraclave_comedia = crea_diccionario_probabilidades_condicionadas(archivos_entrenamiento_comedia, palabras_clave_comedia)

mostrar_diccionario_probabilidades_condicionadas(probabilidad_palabraclave_comedia, "comedia")

P(alemán|comedia) = 	 0.001709
P(escapa|comedia) = 	 0.001709
P(bomba|comedia) = 	 0.001709
P(venganza|comedia) = 	 0.001709
P(problemas|comedia) = 	 0.001709
P(sargento|comedia) = 	 0.001709
P(operación|comedia) = 	 0.001709
P(oficial|comedia) = 	 0.001709
P(enemigo|comedia) = 	 0.001709
P(policía|comedia) = 	 0.001709
P(cabo|comedia) = 	 0.001709
P(carrera|comedia) = 	 0.001709
P(asesino|comedia) = 	 0.001709
P(estadounidenses|comedia) = 	 0.001709
P(misión|comedia) = 	 0.001709
P(investigación|comedia) = 	 0.001709
P(vietnam|comedia) = 	 0.001709
P(teniente|comedia) = 	 0.001709
P(asesinos|comedia) = 	 0.001709
P(combate|comedia) = 	 0.001709
P(ejército|comedia) = 	 0.001709
P(auto|comedia) = 	 0.001709
P(guerra|comedia) = 	 0.001709
P(escapar|comedia) = 	 0.001709
P(seguridad|comedia) = 	 0.001709
P(terrorista|comedia) = 	 0.001709
P(carreras|comedia) = 	 0.001709
P(soldado|comedia) = 	 0.001709
P(grupo|comedia) = 	 0.001709
P(persecución|comedia) = 	 0.001709
P(hospital|comedia) =

In [275]:
probabilidad_palabraclave_terror = crea_diccionario_probabilidades_condicionadas(archivos_entrenamiento_terror, palabras_clave_terror)

mostrar_diccionario_probabilidades_condicionadas(probabilidad_palabraclave_terror, "terror")

P(alemán|terror) = 	 0.000510
P(escapa|terror) = 	 0.000510
P(bomba|terror) = 	 0.000510
P(venganza|terror) = 	 0.000510
P(problemas|terror) = 	 0.000510
P(sargento|terror) = 	 0.000510
P(operación|terror) = 	 0.000510
P(oficial|terror) = 	 0.000510
P(enemigo|terror) = 	 0.000510
P(policía|terror) = 	 0.000510
P(cabo|terror) = 	 0.000510
P(carrera|terror) = 	 0.000510
P(asesino|terror) = 	 0.000510
P(estadounidenses|terror) = 	 0.000510
P(misión|terror) = 	 0.000510
P(investigación|terror) = 	 0.000510
P(vietnam|terror) = 	 0.000510
P(teniente|terror) = 	 0.000510
P(asesinos|terror) = 	 0.000510
P(combate|terror) = 	 0.000510
P(ejército|terror) = 	 0.000510
P(auto|terror) = 	 0.000510
P(guerra|terror) = 	 0.000510
P(escapar|terror) = 	 0.000510
P(seguridad|terror) = 	 0.000510
P(terrorista|terror) = 	 0.000510
P(carreras|terror) = 	 0.000510
P(soldado|terror) = 	 0.000510
P(grupo|terror) = 	 0.000510
P(persecución|terror) = 	 0.000510
P(hospital|terror) = 	 0.000510
P(muerto|terror) = 

In [276]:
probabilidad_palabraclave_bélico = crea_diccionario_probabilidades_condicionadas(archivos_entrenamiento_bélico, palabras_clave_bélico)

mostrar_diccionario_probabilidades_condicionadas(probabilidad_palabraclave_bélico, "bélico")

P(alemán|bélico) = 	 0.001463
P(escapa|bélico) = 	 0.000163
P(bomba|bélico) = 	 0.000163
P(venganza|bélico) = 	 0.000163
P(problemas|bélico) = 	 0.000163
P(sargento|bélico) = 	 0.003576
P(operación|bélico) = 	 0.001138
P(oficial|bélico) = 	 0.001138
P(enemigo|bélico) = 	 0.000975
P(policía|bélico) = 	 0.000163
P(cabo|bélico) = 	 0.000975
P(carrera|bélico) = 	 0.000163
P(asesino|bélico) = 	 0.000163
P(estadounidenses|bélico) = 	 0.001300
P(misión|bélico) = 	 0.001788
P(investigación|bélico) = 	 0.000163
P(vietnam|bélico) = 	 0.001300
P(teniente|bélico) = 	 0.001463
P(asesinos|bélico) = 	 0.000163
P(combate|bélico) = 	 0.002113
P(ejército|bélico) = 	 0.003739
P(auto|bélico) = 	 0.000163
P(guerra|bélico) = 	 0.006502
P(escapar|bélico) = 	 0.000163
P(seguridad|bélico) = 	 0.000163
P(terrorista|bélico) = 	 0.000163
P(carreras|bélico) = 	 0.000163
P(soldado|bélico) = 	 0.001138
P(grupo|bélico) = 	 0.002438
P(persecución|bélico) = 	 0.000163
P(hospital|bélico) = 	 0.000163
P(muerto|bélico) = 

Una vez calculadas todas las probabilidades condicionadas de todas las categorías ya hemos finalizado con este subapartado, pero antes vamos a recordar un par de cosas:

Para acceder a una probabilidad específica símplemente ejecutamos lo siguiente (para el ejemplo obtendremos la __probabilidad de coche__ condicionada a la categoría __acción__, es decir, __P(coche|acción)__):

In [277]:
print("P(coche|acción) = %f" % (probabilidad_palabraclave_acción["coche"]))

P(coche|acción) = 0.002315


Si hubiese una __palabra clave que no pertenece a acción, la probabilidad no sería 0__ ya que estamos usando suavizado. Por ejemplo, podemos comprobarlo con la probabilidad de soldado (que no forma parte de las palabras clave de acción) condicionada a acción (__P(soldado|acción)__):

In [278]:
print("P(soldado|acción) = %f" % (probabilidad_palabraclave_acción["soldado"]))

P(soldado|acción) = 0.000129


### Parte IV-B: Procesamiento de kNN

## Parte V: Salvado del procesamiento en fichero

### Parte V-A: Salvado del procesamiento de Naive Bayes

Dado que el enunciado de la práctica requiere que guardemos el procesado que acabamos que realizar (en la parte IV) para después utilizarlo en la ejecución de los algoritmos, procedemos a ello.

Primero necesitamos crear dos grandes listas: la primera contendrá la cadena relacionada con la probabilidad de que pase algo y la segunda la probabilidad de que pase. Ambas listas deben ser iguales y debemos almacenar toda la información obtenida.

In [279]:
lista_texto = []
lista_valor = []

lista_texto.append("P(acción)")
lista_valor.append(probabilidad_acción)

lista_texto.append("P(comedia)")
lista_valor.append(probabilidad_comedia)

lista_texto.append("P(terror)")
lista_valor.append(probabilidad_terror)

lista_texto.append("P(bélico)")
lista_valor.append(probabilidad_bélico)

for entrada in probabilidad_palabraclave_acción:
    lista_texto.append("P(" + entrada + "|" + "acción" + ")")
    lista_valor.append(probabilidad_palabraclave_acción[entrada])

for entrada in probabilidad_palabraclave_comedia:
    lista_texto.append("P(" + entrada + "|" + "comedia" + ")")
    lista_valor.append(probabilidad_palabraclave_comedia[entrada])

for entrada in probabilidad_palabraclave_terror:
    lista_texto.append("P(" + entrada + "|" + "terror" + ")")
    lista_valor.append(probabilidad_palabraclave_terror[entrada])

for entrada in probabilidad_palabraclave_bélico:
    lista_texto.append("P(" + entrada + "|" + "bélico" + ")")
    lista_valor.append(probabilidad_palabraclave_bélico[entrada])

In [280]:
import csv # Librería que necesitaremos para guardar en formato ".csv".

csvfile = "csv/naive-bayes.csv"
datos_a_guardar = zip(lista_texto, lista_valor)

with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for dato in datos_a_guardar:
        writer.writerow([dato[0]] + [dato[1]])

### Parte V-B: Salvado del procesamiento de kNN

## Parte VI: Ejecución de los algoritmos

En la parte II ya encontramos los ficheros del conjunto de test, que están en su respectiva carpeta, ahora tenemos que procesaros igual que hicimos en la parte III con los ficheros del conjunto de pruebas.

Vamos a definir un método que nos será de utilidad: a partir de un __csv__ dado y de una __cadena a buscar__, devolverá el valor relacionado con dicha celda del csv. Esto nos será útil para, a partir de la cadena de una probabilidad (por ejemplo, _P(bala|acción)_), nos devuelva su probabilidad.

In [281]:
def lee_fichero(nombre_csv):
    with open(nombre_csv, 'rt', encoding="latin-1") as fichero:
        lector = csv.reader(fichero)
        diccionario = dict(lector)
    
    return diccionario

### Parte VI-A: Ejecución de Naive Bayes

El método naive_bayes recibe un archivo y los datos procesados (probabilidades) como parámetros y determina la categoría del archivo pasado como parámetro.

In [282]:
def naive_bayes(archivo, csv):
    cuenta_palabras = cuenta_palabras_desde_archivo(ruta_conjunto_prueba, archivo)
    
    palabras_coincidentes_con_palabras_clave = cuenta_palabras.copy()
    
    # De las palabras que contiene el fichero, desechamos todas las que no coinciden con las palabras clave.
    for palabra in cuenta_palabras:
        if palabra not in palabras_clave:
            del palabras_coincidentes_con_palabras_clave[palabra]
    
    # Abrimos el fichero ".csv" generado para consultar datos en el siguiente paso.
    datos = lee_fichero(csv)
    
    # Ejecutamos el algoritmo en sí.
    resultados = {} # Resultados será un diccionario que contendrá la categoría y la "puntuación" otorgada por el algoritmo a esa categoría (para posteriormente elegir la categoría con el máximo valor).
    
    for categoría in categorías:
        probabilidades_condicionadas_a_multiplicar = []
        for palabra_clave in palabras_coincidentes_con_palabras_clave:
            cadena_a_buscar = "P(" + palabra_clave + "|" + categoría + ")" # Define la cadena que se debe buscar en el archivo. En este caso la probabilidad condicionada a la categoría.
            probabilidades_condicionadas_a_multiplicar.append(float(datos[cadena_a_buscar])) # Añade el valor de la probabilidad condicionada hallada.
        
        cadena_a_buscar = "P(" + categoría + ")" # Define la cadena que se debe buscar en el archivo. En este caso la probabilidad de la categoría.
        probabilidad_categoría = float(datos[cadena_a_buscar])
        
        # Multiplicamos los elementos de la lista de probabilidades condicionadas
        probabilidades_condicionadas_multiplicadas = 1.0
        for elemento in probabilidades_condicionadas_a_multiplicar:
            probabilidades_condicionadas_multiplicadas = probabilidades_condicionadas_multiplicadas * elemento
        
        resultados[categoría] = probabilidades_condicionadas_multiplicadas * probabilidad_categoría # Calcula el coeficiente
    
    result = max(resultados, key=resultados.get) # Devuleve el resultado del algoritmo. En este caso el elemento del diccionario con mayor coeficiente.
    
    return result

__Ejecutamos el algoritmo__ e __imprimimos__ los __resultados__ obtenidos:

In [283]:
for archivo in archivos_prueba:
    print("Tras aplicar el algoritmo Naive-Bayes a [[[%s]]] el resultado es... \t [[[%s]]]" % (archivo.replace(".txt", ""), naive_bayes(archivo, "csv/naive-bayes.csv")))

Tras aplicar el algoritmo Naive-Bayes a [[[Teléfono rojo - Volamos hacia Moscú]]] el resultado es... 	 [[[bélico]]]
Tras aplicar el algoritmo Naive-Bayes a [[[El padrino]]] el resultado es... 	 [[[acción]]]
Tras aplicar el algoritmo Naive-Bayes a [[[La caza del Octubre Rojo]]] el resultado es... 	 [[[comedia]]]
Tras aplicar el algoritmo Naive-Bayes a [[[El tirador]]] el resultado es... 	 [[[comedia]]]


### Parte VI-B: Ejecución de kNN

## Parte VII: Análisis de los resultados

## Parte VIII: Conclusiones